In [1]:
# Imports

from PIL import Image
import tifffile
import os
import math

import numpy as np
import pandas as pd
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Flags

# Use MPS if available?
USE_MPS = False # CNN stuff doesn't work (particularly 3D CNN) with MPS. Only CUDA works.

# Use Lower Quality features (bigger dataset but quite possibly shit data) for training?
USE_LQ_FEATURES = True

# Use ensemble method or use padding?
USE_ENSEMBLE = True

LOAD_WEIGHTS = True # Load pretrained weights if they exist?
TRAIN_MODEL = False # Train model? Can train on pre-loaded weights also
SAVE_WEIGHTS = False # Save weights after training?

ENSEMBLE_WEIGHTS = "ensemble_model_weights.pth" # Weights for ensemble learning model
PADDING_WEIGHTS = "padding_model_weights.pth" # Weights for padding based model

RANDOM_SEED = 42069 # Seed to use for all random operations

In [3]:
# Device setup

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('mps:0') if (USE_MPS and torch.backends.mps.is_available()) else torch.device('cpu')

torch.set_default_device(device)

In [4]:
# DF and Image loading from CSV

hq_csv_file_path = 'full_Table_HIGH_QUAL.csv'
lq_csv_file_path = 'full_Table_STRIPPED_CLEANED.csv'
image_folder_path = "./Images_Of_Networks/tiff/"

df_file_path = lq_csv_file_path if USE_LQ_FEATURES else hq_csv_file_path

# Load the CSV files into pandas DataFrames
df = pd.read_csv(df_file_path)

# Function to load images
def load_tiff_as_tensor(file_name):
    file_path = os.path.join(image_folder_path, file_name)
    if os.path.isfile(file_path):
        image_array = tifffile.imread(file_path)
        # Convert the array to float32 before creating the tensor
        image_array = image_array.astype('float32')
        
        # TEMPORARY!! to ensure there are no images w just one pixel in any dimension
        for i in range (3):
            if (image_array.shape[i] < 2):
                return None
            else:
                print(image_array.shape)
        
        return torch.from_numpy(image_array).float()
    return None

# Update DataFrame 'image' column loading
df['image'] = df['file_name'].apply(load_tiff_as_tensor)

# Drop the 'file_name' column
df.drop('file_name', axis=1, inplace=True)

# Delete rows without images
df = df.dropna(subset=['image'])

(20, 155, 124)
(20, 155, 124)
(20, 155, 124)
(22, 213, 137)
(22, 213, 137)
(22, 213, 137)
(18, 215, 106)
(18, 215, 106)
(18, 215, 106)
(9, 173, 112)
(9, 173, 112)
(9, 173, 112)
(22, 200, 87)
(22, 200, 87)
(22, 200, 87)
(20, 174, 112)
(20, 174, 112)
(20, 174, 112)
(21, 239, 125)
(21, 239, 125)
(21, 239, 125)
(20, 211, 123)
(20, 211, 123)
(20, 211, 123)
(21, 204, 135)
(21, 204, 135)
(21, 204, 135)
(22, 216, 139)
(22, 216, 139)
(22, 216, 139)
(21, 165, 134)
(21, 165, 134)
(21, 165, 134)
(19, 180, 124)
(19, 180, 124)
(19, 180, 124)
(20, 234, 118)
(20, 234, 118)
(20, 234, 118)
(14, 177, 138)
(14, 177, 138)
(14, 177, 138)
(20, 212, 129)
(20, 212, 129)
(20, 212, 129)
(18, 187, 135)
(18, 187, 135)
(18, 187, 135)
(18, 198, 96)
(18, 198, 96)
(18, 198, 96)
(20, 212, 74)
(20, 212, 74)
(20, 212, 74)
(14, 184, 120)
(14, 184, 120)
(14, 184, 120)
(14, 224, 124)
(14, 224, 124)
(14, 224, 124)
(16, 203, 90)
(16, 203, 90)
(16, 203, 90)
(15, 148, 66)
(15, 148, 66)
(15, 148, 66)
(16, 185, 63)
(16, 185, 63)


In [5]:
# Check the largest and smallest x, y, z dimension of each image in the dfs

# Max dimensions will be used to pad all images to be the same size as the max_dimensions
# so that the network has uniform image sizes to train on

# and

# Min dimensions will be used to split all images into multiple images of min_dimension size
# for ensemble learning

def get_image_dimensions(image):
    # Get x and y dimensions from the image size
    x_dim, y_dim, z_dim = image.size(0), image.size(1), image.size(2)
    return x_dim, y_dim, z_dim

# Extract dimensions of all images
dimensions = df['image'].apply(get_image_dimensions)

# Separate into individual lists for x, y, and z
x_dims = [dim[0] for dim in dimensions]
y_dims = [dim[1] for dim in dimensions]
z_dims = [dim[2] for dim in dimensions]

# Find independent max and min dimensions
max_x, max_y, max_z = max(x_dims), max(y_dims), max(z_dims)
min_x, min_y, min_z = min(x_dims), min(y_dims), min(z_dims)

print(f'Largest dimensions: x = {max_x}, y = {max_y}, z = {max_z}')
print(f'Smallest dimensions: x = {min_x}, y = {min_y}, z = {min_z}')

# Use these dimensions to set up the padding correctly
# Note: We need to use max_x, max_y, max_z for padding all images to the same size
max_dims = (max_x, max_y, max_z)
min_dims = (min_x, min_y, min_z)

print("Min dimensions: ", min_dims)
print("Max dimensions: ", max_dims)

# Now split data into train and test set (stratified)
feature_to_predict = 'cell_group'

# Now, perform the split using the binned feature for stratification
train_df, test_df = train_test_split(df, test_size=0.2, random_state=RANDOM_SEED, stratify=df[feature_to_predict])

print("Train Size:", len(train_df))
print("Test Size:", len(test_df))

print("Total Dataset Size:", len(df))

Largest dimensions: x = 22, y = 299, z = 330
Smallest dimensions: x = 2, y = 2, z = 2
Min dimensions:  (2, 2, 2)
Max dimensions:  (22, 299, 330)
Train Size: 4681
Test Size: 1171
Total Dataset Size: 5852


In [6]:
# Custom Dataset class

class CustomDataset(Dataset):
    def __init__(self, dataframe, max_dims, min_dims, use_ensemble):
        self.max_dims = max_dims
        self.min_dims = min_dims
        self.use_ensemble = use_ensemble
        if not use_ensemble:
            self.dataframe = dataframe
        else:
            # Preprocess dataframe to expand sub-images into separate rows
            expanded_data = []
            for idx, row in dataframe.iterrows():
                image_tensor = row['image']  # Assuming this is already a tensor
                sub_images_tensors = self.split_image(image_tensor)
                for sub_img in sub_images_tensors:
                    # Copy numerical features for each sub-image
                    new_row = row.drop('image').to_dict()
                    new_row['image'] = sub_img / 255  # Normalize sub-image
                    expanded_data.append(new_row)
            self.dataframe = pd.DataFrame(expanded_data)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_tensor = row['image']  # This is already a tensor, normalized if in ensemble mode

        # Extract label, the feature to predict
        label = row['cell_group']

        # Extract numerical features
        numerical_features = torch.tensor(row.drop(['image', 'cell_group'], errors='ignore').values.astype(np.float32))

        if not self.use_ensemble:
            # If not in ensemble mode, pad and normalize the image tensor
            image_tensor = self.pad_image(image_tensor) / 255
        else:
            # If in ensemble mode, image_tensor is already a normalized sub-image tensor,
            # so there's no need for additional processing here.
            pass

        return image_tensor, numerical_features, label

    def pad_image(self, image):
        # Assuming image is a tensor
        current_x, current_y, current_z = image.shape
        max_x, max_y, max_z = self.max_dims

        # Calculate padding
        pad_x = max_x - current_x
        pad_y = max_y - current_y
        pad_z = max_z - current_z

        # Apply padding
        # PyTorch's functional pad expects padding in reverse order of dimensions and for all dimensions
        image_padded = F.pad(image, (0, pad_z, 0, pad_y, 0, pad_x), 'constant', 0)
        return image_padded

    def split_image(self, image):
        min_x, min_y, min_z = self.min_dims
        sub_images = []
        for x in range(0, image.shape[0] - min_x + 1, min_x):
            for y in range(0, image.shape[1] - min_y + 1, min_y):
                for z in range(0, image.shape[2] - min_z + 1, min_z):
                    sub_img = image[x:x+min_x, y:y+min_y, z:z+min_z]
                    sub_images.append(sub_img)
        return sub_images


In [7]:
# Define the model

class CustomCNN(nn.Module):
    def __init__(self, num_numerical_features, image_dims, use_ensemble):
        super(CustomCNN, self).__init__()
        
        self.use_ensemble = use_ensemble
        
        # Unpack image dimensions
        self.x_dim, self.y_dim, self.z_dim = image_dims
        
        # Convolutional layers
        self.conv1 = nn.Conv3d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv3d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv3d(32, 64, kernel_size=3, stride=1, padding=1)
        
        # Adaptive pool to make the output size fixed before the fully connected layers
        self.adaptive_pool = nn.AdaptiveAvgPool3d((5, 5, 5))
        
        # Calculate the size for the first fully connected layer
        fc1_input_size = 5 * 5 * 5 * 64 + num_numerical_features
        
        # Fully connected layers
        self.fc1 = nn.Linear(fc1_input_size, 128)
        self.fc2 = nn.Linear(128, 1)  # Output size is 1 for binary classification
        
    def forward(self, image_tensors, numerical_features):
        if self.use_ensemble:
            predictions = []
            for image_tensor in image_tensors:
                # Add both batch and channel dimensions for 3D grayscale image
                image_tensor = image_tensor.unsqueeze(1)  # Now it's [Batch Size, 1, D, H, W]
                features = self.extract_features(image_tensor)
                combined_features = torch.cat((features, numerical_features), dim=1)
                x = F.relu(self.fc1(combined_features))
                prediction = torch.sigmoid(self.fc2(x))
                predictions.append(prediction)
            final_prediction = torch.max(torch.stack(predictions), dim=0)[0]
            
            # THE ABOVE BIT FOR ENSEMBLE PROLLY NEEDS MORE WORK! PROBABLY DOESN'T QUITE WORK YET
        else:
            # Add both batch and channel dimensions for 3D grayscale image
            image_tensors = image_tensors.unsqueeze(1)  # Now it's [Batch Size, 1, D, H, W]
            features = self.extract_features(image_tensors)
            combined_features = torch.cat((features, numerical_features), dim=1)
            x = F.relu(self.fc1(combined_features))
            final_prediction = torch.sigmoid(self.fc2(x))
        return final_prediction
    
    def extract_features(self, x):
        # Convolutional layers with ReLU and MaxPool
        x = F.max_pool3d(F.relu(self.conv1(x)), 2)
        x = F.max_pool3d(F.relu(self.conv2(x)), 2)
        x = F.max_pool3d(F.relu(self.conv3(x)), 2)
        
        # Adaptive pooling
        x = self.adaptive_pool(x)
        
        # Flatten
        x = torch.flatten(x, 1)
        return x

In [8]:
def train_model(model, train_loader, test_loader, num_epochs=10, device=device):
    print("Starting training...")
    
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for data in train_loader:
            # Unpack data
            if model.use_ensemble:
                images, numerical_features, labels = zip(*data)
                images = [img.to(device) for sublist in images for img in sublist]  # Flatten list of lists
                numerical_features = torch.cat(numerical_features).to(device)
                labels = torch.cat(labels).to(device)
            else:
                images, numerical_features, labels = data
                images, numerical_features, labels = images.to(device), numerical_features.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            if model.use_ensemble:
                outputs = torch.stack([model(img.unsqueeze(0), numerical_features[i].unsqueeze(0)) for i, img in enumerate(images)])
                outputs = outputs.mean(dim=0).squeeze()  # Aggregate by averaging
            else:
                outputs = model(images, numerical_features).squeeze()

            # Calculate loss and backpropagate
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

        # Evaluation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:
                # Unpack data
                if model.use_ensemble:
                    images, numerical_features, labels = zip(*data)
                    images = [img.to(device) for sublist in images for img in sublist]  # Flatten list of lists
                    numerical_features = torch.cat(numerical_features).to(device)
                    labels = torch.cat(labels).to(device)
                else:
                    images, numerical_features, labels = data
                    images, numerical_features, labels = images.to(device), numerical_features.to(device), labels.to(device)

                # Forward pass
                if model.use_ensemble:
                    outputs = torch.stack([model(img.unsqueeze(0), numerical_features[i].unsqueeze(0)) for i, img in enumerate(images)])
                    outputs = outputs.max(dim=0)[0].squeeze()  # Aggregate by taking max
                else:
                    outputs = model(images, numerical_features).squeeze()

                predicted = outputs > 0.5
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Accuracy on test set: {accuracy}%')

In [9]:
# Create CustomDatasets
train_dataset = CustomDataset(train_df, max_dims, min_dims, USE_ENSEMBLE)
test_dataset = CustomDataset(test_df, max_dims, min_dims, USE_ENSEMBLE)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model = CustomCNN(num_numerical_features=len(train_df.drop(['image', 'cell_group'], axis=1).columns),
                  image_dims=min_dims if USE_ENSEMBLE else max_dims,
                  use_ensemble=USE_ENSEMBLE)
model.to(device)

# Load pretrained weights if available and desired
weights_path = ENSEMBLE_WEIGHTS if USE_ENSEMBLE else PADDING_WEIGHTS
if LOAD_WEIGHTS and os.path.isfile(weights_path):
    model.load_state_dict(torch.load(weights_path, map_location=device))

# Train the model if needed
if TRAIN_MODEL:
    train_model(model, train_loader, test_loader, num_epochs=10, device=device)
    if SAVE_WEIGHTS:
        torch.save(model.state_dict(), weights_path)

In [ ]:
model.eval()
accuracies = []  # This will store True for correct predictions and False for incorrect predictions
pixel_counts_accuracies = []

with torch.no_grad():
    for idx, row in test_df.iterrows():
        # Create a single-row DataFrame from the current row
        single_row_df = pd.DataFrame([row])

        # Initialize a CustomDataset instance with this single-row DataFrame
        single_row_dataset = CustomDataset(single_row_df, max_dims, min_dims, USE_ENSEMBLE)

        # Create a DataLoader for this dataset with a batch size of 1
        single_row_loader = DataLoader(single_row_dataset, batch_size=1, shuffle=False)

        # Run inference using this DataLoader
        for images, numerical_features, labels in single_row_loader:
            images, numerical_features, labels = images.to(device), numerical_features.to(device), labels.to(device)

            outputs = model(images, numerical_features).squeeze()
            predicted = outputs > 0.5  # Threshold for binary classification

            # Determine the correctness of each prediction
            is_correct = (predicted == labels).item()
            accuracies.append(is_correct)

        # Determine the pixel count for the current datapoint
        pixel_count = math.prod(row['image'].shape)
        pixel_counts_accuracies.append(pixel_count)

In [ ]:
model.eval()
recalls = []  # This will store True for correct predictions and False for incorrect predictions
pixel_counts_recalls = []

with torch.no_grad():
    for idx, row in train_df.iterrows():
        # Create a single-row DataFrame from the current row
        single_row_df = pd.DataFrame([row])

        # Initialize a CustomDataset instance with this single-row DataFrame
        single_row_dataset = CustomDataset(single_row_df, max_dims, min_dims, USE_ENSEMBLE)

        # Create a DataLoader for this dataset with a batch size of 1
        single_row_loader = DataLoader(single_row_dataset, batch_size=1, shuffle=False)

        # Run inference using this DataLoader
        for images, numerical_features, labels in single_row_loader:
            images, numerical_features, labels = images.to(device), numerical_features.to(device), labels.to(device)

            outputs = model(images, numerical_features).squeeze()
            predicted = outputs > 0.5  # Threshold for binary classification

            # Determine the correctness of each prediction
            is_correct = (predicted == labels).item()
            recalls.append(is_correct)

        # Determine the pixel count for the current datapoint
        pixel_count = math.prod(row['image'].shape)
        pixel_counts_recalls.append(pixel_count)

In [ ]:
fig = go.Figure()

# Add points for correct predictions
fig.add_trace(go.Scatter(
    x=[pixel_counts_recalls[i] for i in range(len(recalls)) if recalls[i]],
    y=[1 for i in range(len(recalls)) if recalls[i]],  # Use 1 as a dummy value to represent correct predictions
    mode='markers',
    name='Correct',
    marker=dict(color='Blue', size=12, line=dict(color='MediumBlue', width=2))
))

# Add points for incorrect predictions
fig.add_trace(go.Scatter(
    x=[pixel_counts_recalls[i] for i in range(len(recalls)) if not recalls[i]],
    y=[0 for i in range(len(recalls)) if not recalls[i]],  # Use 0 as a dummy value to represent incorrect predictions
    mode='markers',
    name='Incorrect',
    marker=dict(color='Red', size=12, line=dict(color='DarkRed', width=2))
))

fig.update_layout(
    title='Correctness vs. Number of Pixels in Image (Recall)',
    xaxis_title='Number of Pixels',
    xaxis_type='log',  # Set the x-axis to a logarithmic scale
    yaxis_title='Correctness',
    yaxis=dict(tickvals=[0, 1], ticktext=['Incorrect', 'Correct'])
)

fig.show()


In [ ]:
log_bins = np.geomspace(min(pixel_counts_recalls), max(pixel_counts_recalls), num=21)

# Correct predictions
correct_pixel_counts = [count for count, recall in zip(pixel_counts_recalls, recalls) if recall]

# Incorrect predictions
incorrect_pixel_counts = [count for count, recall in zip(pixel_counts_recalls, recalls) if not recall]

# Digitize/bucket the pixel counts into these bins for correct and incorrect predictions
correct_binned = np.digitize(correct_pixel_counts, bins=log_bins)
incorrect_binned = np.digitize(incorrect_pixel_counts, bins=log_bins)

# Count occurrences in each bin
correct_counts = [list(correct_binned).count(i) for i in range(1, len(log_bins))]
incorrect_counts = [list(incorrect_binned).count(i) for i in range(1, len(log_bins))]

# Create the bar graph with uniform width bars in log space
fig = go.Figure()

# Helper function to convert bins to strings for better labeling
def format_bins(bins):
    labels = []
    for i in range(len(bins)-1):
        labels.append(f"{bins[i]:.2f} - {bins[i+1]:.2f}")
    return labels

bin_labels = format_bins(log_bins)

# Add correct predictions bars
fig.add_trace(go.Bar(
    x=bin_labels,
    y=correct_counts,
    name='Correct',
    marker_color='Blue'
))

# Add incorrect predictions bars
fig.add_trace(go.Bar(
    x=bin_labels,
    y=incorrect_counts,
    name='Incorrect',
    marker_color='Red'
))

# Update the layout to display the bin ranges correctly
fig.update_layout(
    title='Correct and Incorrect Predictions in Recall',
    xaxis_title='Number of Pixels (Bin Ranges)',
    xaxis={'type':'category'},
    yaxis_title='Count',
    barmode='group'
)

fig.show()

accuracy = sum(recalls) / len(recalls)

print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.54


In [ ]:
fig = go.Figure()

# Add points for correct predictions
fig.add_trace(go.Scatter(
    x=[pixel_counts_accuracies[i] for i in range(len(accuracies)) if accuracies[i]],
    y=[1 for i in range(len(accuracies)) if accuracies[i]],  # Use 1 as a dummy value to represent correct predictions
    mode='markers',
    name='Correct',
    marker=dict(color='Blue', size=12, line=dict(color='MediumBlue', width=2))
))

# Add points for incorrect predictions
fig.add_trace(go.Scatter(
    x=[pixel_counts_accuracies[i] for i in range(len(accuracies)) if not accuracies[i]],
    y=[0 for i in range(len(accuracies)) if not accuracies[i]],  # Use 0 as a dummy value to represent incorrect predictions
    mode='markers',
    name='Incorrect',
    marker=dict(color='Red', size=12, line=dict(color='DarkRed', width=2))
))

fig.update_layout(
    title='Correctness vs. Number of Pixels in Image (Accuracy)',
    xaxis_title='Number of Pixels',
    xaxis_type='log',  # Set the x-axis to a logarithmic scale
    yaxis_title='Correctness',
    yaxis=dict(tickvals=[0, 1], ticktext=['Incorrect', 'Correct'])
)

fig.show()


In [ ]:
correct_pixel_counts = [count for count, accuracy in zip(pixel_counts_accuracies, accuracies) if accuracy]

# Incorrect predictions
incorrect_pixel_counts = [count for count, accuracy in zip(pixel_counts_accuracies, accuracies) if not accuracy]

# Create logarithmic bins with uniform width in log space
log_bins = np.geomspace(min(pixel_counts_accuracies), max(pixel_counts_accuracies), num=21)

# Digitize/bucket the pixel counts into these bins for correct and incorrect predictions
correct_binned = np.digitize(correct_pixel_counts, bins=log_bins)
incorrect_binned = np.digitize(incorrect_pixel_counts, bins=log_bins)

# Count occurrences in each bin
correct_counts = [list(correct_binned).count(i) for i in range(1, len(log_bins))]
incorrect_counts = [list(incorrect_binned).count(i) for i in range(1, len(log_bins))]

# Create the bar graph with uniform width bars in log space
fig = go.Figure()

# Helper function to convert bins to strings for better labeling
def format_bins(bins):
    labels = []
    for i in range(len(bins)-1):
        labels.append(f"{bins[i]:.2f} - {bins[i+1]:.2f}")
    return labels

bin_labels = format_bins(log_bins)

# Add correct predictions bars
fig.add_trace(go.Bar(
    x=bin_labels,
    y=correct_counts,
    name='Correct',
    marker_color='Blue'
))

# Add incorrect predictions bars
fig.add_trace(go.Bar(
    x=bin_labels,
    y=incorrect_counts,
    name='Incorrect',
    marker_color='Red'
))

# Update the layout to display the bin ranges correctly
fig.update_layout(
    title='Correct and Incorrect Predictions in Logarithmic Bins (Accuracy)',
    xaxis_title='Number of Pixels (Bin Ranges)',
    xaxis={'type':'category'},
    yaxis_title='Count',
    barmode='group'
)

# Calculate accuracy
accuracy = sum(accuracies) / len(accuracies)
print(f'Accuracy: {accuracy:.2%}')

# Show the figure
fig.show()

Accuracy: 52.52%


In [ ]:
# The features in the dataset are:

for x in train_df.columns:
    print(x)

x
y
z
node_x
line_id
point_id
degree_x
vol_cc_x
avg_PK_Of_element_x
element_connectivity_x
Unnamed: 0_y
cc_length_(um)
cc_vol_from_img_(um3)
branches
nodes
edges
cc_average_degree_excludeFreeEnds
cc_max_PK
cc_average_connectivity
diameter
element_length_(um)
element_average_width
element_Volume_Voxel
x_x
y_x
z_x
width_(um)
x_pixel
y_pixel
z_pixel
distance
x_y
y_y
z_y
node_y
degree_y
cc_y
vol_cc_y
avg_PK_Of_element_y
element_connectivity_y
coeff_of_variance_cc
coeff_of_variance_line_id
density
pathLength
clustering_coefficient
matching_index
cc_center_x
cc_center_y
cc_center_z
center_x
center_y
center_z
distance_to_center
order
Angle_Flag
1hot_folder_name_0
1hot_folder_name_1
1hot_folder_name_2
1hot_folder_name_3
1hot_folder_name_4
1hot_folder_name_5
1hot_folder_name_6
1hot_folder_name_7
1hot_folder_name_8
1hot_folder_name_9
1hot_folder_name_10
1hot_folder_name_11
1hot_folder_name_12
1hot_folder_name_13
1hot_folder_name_14
1hot_folder_name_15
1hot_folder_name_16
1hot_folder_name_17
1hot